In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time
import json
import plotly.graph_objects as go

from sklearn.preprocessing import StandardScaler

In [ ]:
energy_data = pd.read_csv('/home/nissatech/Documents/Double Machine Learning/Raw Data/energy_data.csv')

In [ ]:
env_data = pd.read_csv('/home/nissatech/Documents/Double Machine Learning/Raw Data/environment_data.csv')

<h1>1 Exploratory data analysis</h1>

<h2>1.1 Environment data </h2>

In [ ]:
env_data.head()

In [ ]:
env_data.columns

<h3>1.1.1 Environment data indexing</h3>

In [ ]:
df_indexed_env_data = env_data.copy()
df_indexed_env_data['timestamp'] = pd.to_datetime(df_indexed_env_data['timestamp'])
df_indexed_env_data.index = df_indexed_env_data['timestamp']
del df_indexed_env_data['timestamp']

<h3>1.1.2 Environment data visualization</h3>

In [ ]:
for column in df_indexed_env_data.columns:
    df_indexed_env_data[column].plot(figsize = (14, 8), title = column)
    plt.show()

<h2>1.2 Energy data</h2>

In [ ]:
energy_data.shape

In [ ]:
energy_data.head()

<h3>1.2.1 Energy data indexing</h3>

In [ ]:
df_indexed_energy_data = energy_data.copy()
df_indexed_energy_data['timestamp'] = pd.to_datetime(df_indexed_energy_data['timestamp'])
df_indexed_energy_data.index = df_indexed_energy_data['timestamp']
del df_indexed_energy_data['timestamp']

<h3> 1.2.2 Energy data visualization </h3>

In [ ]:
for column in df_indexed_energy_data.columns:
    df_indexed_energy_data[column].plot(figsize = (14, 8), title = column)
    plt.show()

<h1>2 Data preprocessing</h1>

<h2>2.1 Energy data</h2>

<h3>2.1.1 Checking for NaN values in energy data</h3>

In [ ]:
df_indexed_energy_data.isna().any().any()

<h3>2.1.2 Removing highly correlated attributes in energy data</h3>

In [ ]:
corr_matrix = df_indexed_energy_data.corr()

In [ ]:
# Create a larger figure
fig, ax = plt.subplots(figsize=(14,12))

# Plot the correlation matrix as a heatmap
sns.heatmap(corr_matrix,
            cmap='coolwarm',
            annot=True,
            ax=ax,
            annot_kws={"fontsize":8},
            xticklabels=corr_matrix.columns,
            yticklabels=corr_matrix.columns)

# Set the font size of the x and y axis tick labels
ax.tick_params(axis='x', labelsize=8)
ax.tick_params(axis='y', labelsize=8)

plt.show()

In [ ]:
columns_to_drop = ['Active Power A average [kW]','Active Power B average [kW]', 'Active Power C average [kW]', 
                   'Current B average [A]', 'Current C average [A]',
                   'Reactive Power A average [kVAr]','Reactive Power B average [kVAr]', 'Reactive Power C average [kVAr]', 
                   'THDI A average [%]', 'THDI B average [%]', 'THDI C average [%]',
                   'THDU B average [%]', 'THDU C average [%]',
                   'Power Factor A average', 'Power Factor B average', 'Power Factor C average',
                   'Voltage B average [V]',
                  ]
df_energy_data_without_high_correlated = df_indexed_energy_data.drop(columns_to_drop, axis = 1)

In [ ]:
# Calculate the correlation matrix
corr_matrix = df_energy_data_without_high_correlated.corr()

# Create a larger figure
fig, ax = plt.subplots(figsize=(14,12))

# Plot the correlation matrix as a heatmap
sns.heatmap(corr_matrix,
            cmap='coolwarm',
            annot=True,
            ax=ax,
            annot_kws={"fontsize":8},
            xticklabels=corr_matrix.columns,
            yticklabels=corr_matrix.columns)

# Set the font size of the x and y axis tick labels
ax.tick_params(axis='x', labelsize=8)
ax.tick_params(axis='y', labelsize=8)

plt.show()

<h2>2.2 Environmental data </h2>

<h3>2.2.1 Checking for NaN values in environment data </h3>

In [ ]:
df_indexed_env_data.isna().any().any()

In [ ]:
num_rows_with_nan = df_indexed_env_data.isna().any(axis=1).sum()
num_rows_with_nan

In [ ]:
df_indexed_env_data.dropna(inplace=True)

In [ ]:
df_indexed_env_data.isna().any().any()

In [ ]:
df_indexed_energy_data.shape

In [ ]:
df_indexed_env_data.shape

<h2>2.3 Resampling (at a frequency of 1Hz)</h2>

<h3>2.3.1 Resampling energy data</h3>

In [ ]:
df_resampled_energy = df_indexed_energy_data.resample('1S').mean()
df_interpolated_energy = df_resampled_energy.interpolate()

<h3>2.3.2 Resampling environmental data</h3>

In [ ]:
df_resampled_env = df_indexed_env_data.resample('1S').mean()
df_interpolated_env = df_resampled_env.interpolate()

In [ ]:
df_interpolated_env.shape

In [ ]:
for column in df_interpolated_env.columns:
    df_interpolated_env[column].plot(figsize = (14, 8), title = column)
    plt.show()

In [ ]:
df_interpolated_env.isna().any().any()

<h2>2.4 Index alignment</h2>

In [ ]:
print(df_interpolated_energy.shape)
print(df_interpolated_energy.index[0])
print(df_interpolated_energy.index[-1])

In [ ]:
print(df_interpolated_env.shape)
print(df_interpolated_env.index[0])
print(df_interpolated_env.index[-1])

<b>After resampling, the indices for energy and environment data are not aligned, so alignment is necessary.</b>

In [ ]:
# align
df_interpolated_energy = df_interpolated_energy.iloc[:-2]
df_interpolated_env = df_interpolated_env.iloc[2:]

# check indices again
print(df_interpolated_energy.index[0])
print(df_interpolated_energy.index[-1])

print(df_interpolated_env.index[0])
print(df_interpolated_env.index[-1])

<h1>3 Creating new datasets</h1> 

<h2>3.1 Monday dataset</h2>

In [ ]:
# One day, phase A
# Choosing features
columns_to_drop = ['Active Power B average [kW]','Active Power C average [kW]',
                   'Current B average [A]', 'Current C average [A]',
                   'Power Factor B average', 'Power Factor C average',
                   'Reactive Power B average [kVAr]', 'Reactive Power C average [kVAr]', 
                   'THDI B average [%]', 'THDI C average [%]',
                   'THDU B average [%]', 'THDU C average [%]',
                   'Voltage B average [V]', 'Voltage C average [V]'
                  ]
df_phase_A = df_interpolated_energy.drop(columns = columns_to_drop)

# Choosing day
day = pd.Timestamp('2023-04-03')

In [ ]:
mask_day = (df_phase_A.index.date == day.date())
df_monday = df_phase_A.loc[mask_day]

In [ ]:
# The machine only works when current is different from 0, so we shorten the data set to working hours.
mask_working_hours = (df_phase_A.index.date == day.date()) & (df_phase_A.index.hour >= 7) & (df_phase_A.index.hour < 19)
df_working_hours = df_phase_A.loc[mask_working_hours]

In [ ]:
# Due to the demanding calculation of the model it will be taken only before noon.
mask_before_noon = (df_phase_A.index.date == day.date()) & (df_phase_A.index.hour >= 7) & (df_phase_A.index.hour < 12)
df_before_noon = df_phase_A.loc[mask_before_noon]

In [ ]:
# plt.figure(figsize = (14, 8))
# df_one_day_A['Current A average [A]'].plot(title = 'Current A average [A]')

# plt.figure(figsize = (14, 8))
# df_working_hours['Current A average [A]'].plot(title = 'Current A average [A]')

# plt.figure(figsize = (14, 8))
# df_before_noon['Current A average [A]'].plot(title = 'Current A average [A]')

In [ ]:
start_time = pd.Timestamp('2023-04-03 11:00:00')
end_time = pd.Timestamp('2023-04-03 14:30:00')

mask_two_hours = (df_phase_A.index >= start_time) & (df_phase_A.index < end_time)
df_two_hours = df_phase_A.loc[mask_two_hours]

In [ ]:
df_two_hours = df_two_hours[df_two_hours['THDI A average [%]'] > 0]
df_energy_3 = df_two_hours

In [ ]:
df_two_hours.head(1)

In [ ]:
df_two_hours.tail(1)

<h2>3.2 Helper Functions</h2>

features (X), 
treatment variable (T), 
outcome variable (Y)

In [ ]:
# create datasets for all possible combinations of control variables
def create_all_Xs(df):
    Xs = []
    for column in df.columns:
        X = df.drop(column, axis=1)
        Xs.append(X)
    return Xs

In [ ]:
def ndarray_to_list(obj):
    if isinstance(obj, np.ndarray):
        return obj.tolist()
    raise TypeError

In [ ]:
def visualize_models(environmental_models):
    
    for env_parameter_name, env_models in environmental_models.items():
        for energy_parameter_name, model in env_models.items():
    
            # Create a histogram of the CATE estimates - Conditional Average Treatment Effect
            fig = go.Figure(data=[go.Histogram(x=model['pnt_effect'], nbinsx=30)])
            
            fig.update_layout(
                title_text='Distribution of Estimated CATE for Feature: {}, Outcome: {}'.format(energy_parameter_name, env_parameter_name), 
                xaxis_title_text='Estimated CATE', 
                yaxis_title_text='Frequency', 
            )
            
            fig.show()

In [ ]:
def persist(env_models, filename):
    with open('/home/nissatech/Documents/Double Machine Learning/Models/' + filename + '.json', 'w') as f:
        json.dump(env_models, f, default=ndarray_to_list)

In [ ]:
def load_models(filename):
    with open(filename, 'r') as f:
        models = json.load(f)
    return models

In [ ]:
def find_causality(df_energy, df_env, est, alpha=0.05, verbose=False):
    """
    Trains a provided estimator on multiple treatments and outcomes, 
    recording effects and coefficients.
    
    Args:
        df_energy (DataFrame): DataFrame of treatment variables.
        df_env (DataFrame): DataFrame of outcome variables.
        est: The estimator to use for causal inference.
        alpha (float): The significance level for confidence intervals.
        verbose (bool): Whether to print progress updates.
    
    Returns:
        dict: Nested dictionary of model outputs.
    """
    
    Xs = create_all_Xs(df_energy)

    environmental_models = {}
    
    for (outcome_name, Y) in df_env.iteritems():
        if verbose: print("Outcome: " + outcome_name)
        fitted_models = {}
        for X, (treatment_name, T) in zip(Xs, df_energy.iteritems()):
            if verbose: print("\tTreatment: " + treatment_name)
            start_time = time.time()
            est.fit(Y, T, X=X)
            end_time = time.time()
            training_time = end_time - start_time
    
            model_outputs = {
                'pnt_effect': est.const_marginal_effect(X),
                'lb_effect': est.const_marginal_effect_interval(X, alpha)[0],
                'ub_effect': est.const_marginal_effect_interval(X, alpha)[1],
                'pnt_coef': est.coef_,
                'lb_coef': est.coef__interval(alpha=.05)[0],
                'ub_coef': est.coef__interval(alpha=.05)[1],
            }
    
            fitted_models[treatment_name] = model_outputs
            if verbose: print(f"\tTraining time: {training_time} seconds") 
                
        environmental_models[outcome_name] = fitted_models
        
    return environmental_models

<h1>4 Econ ML</h1>

In [ ]:
# pip install econml

In [ ]:
import econml

<h2>4.1 Data Preparation</h2>

In [ ]:
Xs = create_all_Xs(df_before_noon)

<h2>4.2 Model </h2>

We choose Orthogonal Random Forest (ORF), because it is intended for use cases when both treatment and output variables are continuous.

In [ ]:
from econml.dml import LinearDML
from sklearn.ensemble import RandomForestRegressor

<h3>4.2.1 Create and train the model</h3>

model_y - outcome model <p>
model_t - treatment model

In [ ]:
est = LinearDML(model_y=RandomForestRegressor(),
                model_t=RandomForestRegressor())

In [ ]:
# Here, the effects of energy data on CO2 are examined
fitted_models = {}

for X, (column_name, T) in zip(Xs, df_before_noon.iteritems()):
    start_time = time.time()
    est.fit(Y, T, X=X)
    end_time = time.time()
    training_time = end_time - start_time
    
    model_outputs = {
        'pnt_effect': est.const_marginal_effect(X),
        'lb_effect': est.const_marginal_effect_interval(X, alpha=.05)[0],
        'ub_effect': est.const_marginal_effect_interval(X, alpha=.05)[1],
        'pnt_coef': est.coef_,
        'lb_coef': est.coef__interval(alpha=.05)[0],
        'ub_coef': est.coef__interval(alpha=.05)[1],
        'training_time': training_time
    }

    fitted_models[column_name] = model_outputs
    
    print(f"Training time: {training_time} seconds")  

<h3>4.2.2 Model outputs visualization</h3>

<h4>4.2.2.1 CATE - Conditional Average Treatment Effect</h4>

In [ ]:
for model in fitted_models:

    # Create a histogram of the CATE estimates - Conditional Average Treatment Effect
    fig = go.Figure(data=[go.Histogram(x=fitted_models[model]['pnt_effect'], nbinsx=30)])
    
    fig.update_layout(
        title_text='Distribution of Estimated CATE for Feature: {}, Outcome:'.format(model), , 
        xaxis_title_text='Estimated CATE', 
        yaxis_title_text='Frequency', 
    )
    
    fig.show()

<h4>4.2.2.2 ATE - Average Treatment Effect</h4>

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Initialize a dictionary to store the ATEs
ate_dict = {}

for model in fitted_models:
    
    # Estimate the CATE
    cate_estimates = fitted_models[model]['pnt_effect']

    # Estimate the ATE by taking the average of the CATE estimates
    ate_estimate = np.mean(cate_estimates)

    # Store the ATE in the dictionary
    ate_dict[model] = ate_estimate

# Convert the dictionary to a DataFrame
ate_df = pd.DataFrame(list(ate_dict.items()), columns=['Model', 'ATE'])

# Create a bar plot of the ATEs
plt.figure(figsize=(10, 6))
plt.bar(ate_df['Model'], ate_df['ATE'], color='blue')
plt.ylabel('Average Treatment Effect')
plt.title('Average Treatment Effects for Different Models')
plt.xticks(rotation=90)  # Rotate the x-axis labels for better readability
plt.show()

<h1>5 Three hours data set</h1>

In [ ]:
df_env_3 = df_interpolated_env.loc[df_two_hours.index[0]:df_two_hours.index[-1]]

In [ ]:
environmental_models_raw = find_causality(df_energy_3, df_env_3)

In [ ]:
visualize_models(environmental_models_raw)

In [ ]:
persist(environmental_models_raw, "Raw")

<h1>6 Normalized data set</h1>

In [ ]:
scaler = StandardScaler()
df_energy_3_normalized = pd.DataFrame(scaler.fit_transform(df_energy_3), 
                                      columns=df_energy_3.columns, 
                                      index=df_energy_3.index)

df_env_3_normalized = pd.DataFrame(scaler.fit_transform(df_env_3), 
                                      columns=df_env_3.columns, 
                                      index=df_env_3.index)

In [ ]:
env_models_normalized = find_causality(df_energy_3_normalized, df_env_3_normalized)

In [ ]:
visualize_models(env_models_normalized)

In [ ]:
persist(env_models_normalized, "Normalized")

<h1>7 Differentiated data set</h1>

In [ ]:
df_energy_3_diff = df_energy_3.diff()
df_env_3_diff = df_env_3.diff()

df_energy_3_diff = df_energy_3_diff.dropna()
df_env_3_diff = df_env_3_diff.dropna()

In [ ]:
df_energy_3_diff.columns

In [ ]:
df_env_3_diff.columns

In [ ]:
env_models_diff = find_causality(df_energy_3_diff, df_env_3_diff)

In [ ]:
visualize_models(env_models_diff)

In [ ]:
persist(env_models_diff, "Diff")

<h1>8 Include Environment Variables in W parameter</h1>

In [ ]:
X = df_energy_3_diff.copy().drop('Current A average [A]', axis=1)
T = df_energy_3_diff['Current A average [A]']
Y = df_env_3_diff['CO2']
W = df_env_3_diff.copy().drop('CO2', axis = 1)

# estimator here
est = LinearDML(model_y=RandomForestRegressor(),
                model_t=RandomForestRegressor())

est.fit(Y, T, X=X, W=W)
model_outputs = {
    'pnt_effect': est.const_marginal_effect(X),
    'lb_effect': est.const_marginal_effect_interval(X, alpha=0.05)[0],
    'ub_effect': est.const_marginal_effect_interval(X, alpha=0.05)[1],
    'pnt_coef': est.coef_,
    'lb_coef': est.coef__interval(alpha=.05)[0],
    'ub_coef': est.coef__interval(alpha=.05)[1],
}

In [ ]:
fig = go.Figure(data=[go.Histogram(x=model_outputs['pnt_effect'], nbinsx=30)])          
fig.update_layout(
    title_text='Distribution of Estimated CATE for Treatment: {}, Outcome: {}'.format('Current A average [A]', 'CO2'), 
    xaxis_title_text='Estimated CATE', 
    yaxis_title_text='Frequency', 
)
fig.show()

<h1>9 Visualize residuals</h1>

In [ ]:
# TODO: Ispraviti ovaj Approach pošto drugi deo nije tačan
# Potrebno je obe vrste reziduala samostalno izračunati
# Takođe proveriti šta je u tom slučaju sa randomnes

In [ ]:
# First, compute the residuals for the treatment
# Train a model to predict T from X and W using RandomForestRegressor
model_t = RandomForestRegressor()
model_t.fit(pd.concat([X, W], axis=1), T)
predicted_T = model_t.predict(pd.concat([X, W], axis=1))

residual_T = T - predicted_T

# Now, compute the residuals for the outcome
predicted_effect = est.effect(X).flatten()
residual_Y = Y - predicted_effect

In [ ]:
# Scatter plot of the residuals
plt.figure(figsize=(10, 6))
plt.scatter(residual_T, residual_Y, alpha=0.5)
plt.xlabel('Residual Treatment (ml_m)')
plt.ylabel('Residual Outcome (ml_l)')
plt.title('Residuals from DML')
plt.show()

<h1>10 Visualize confidence intervals - EconML git</h1>